In [1]:
from PIL import Image
import numpy as np
import lightgbm as lgb
import skimage.filters as skfilter
import pandas as pd
import os

from copy import deepcopy

from sklearn.preprocessing import OneHotEncoder, LabelEncoder


In [2]:
# load data

train_dir = 'train'
test_dir = 'test'


def open_image(image):
    
    tmp_im_custom = image[0]
    
    for j in range(1, len(image)):
        tmp_im_custom = np.append(tmp_im_custom, image[j])
        
    return tmp_im_custom

def get_data(dir_name):
    
    x = []
    y = []

    for emotion in ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']:
        

        for file_name in os.listdir('./archive-1/'+str(dir_name)+'/'+str(emotion)):

            tmp_im = np.array(Image.open('./archive-1/'+str(dir_name)+'/'+str(emotion)+'/'+str(file_name)))
            
            #tmp_im_custom = open_image(tmp_im)
            
            x.append(tmp_im)
            
            #x.append(tmp_im_custom)

            y.append(emotion)
            
    y_custom = LabelEncoder().fit_transform(np.array(y).reshape(-1,1))
           
    return x, y_custom



train_raw_x, train_raw_y = get_data(train_dir)

test_raw_x, test_raw_y = get_data(test_dir)
    




/Users/hibiki/miniforge3/envs/my-env/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [3]:
# preprocess data

filter_name = ["sobel", "scharr", "prewitt", "roberts"]
filter_inst = [skfilter.sobel, skfilter.scharr, skfilter.prewitt, skfilter.roberts]


filtered_images_train = []
filtered_images_test = []

def use_filters(image_data, filt):
    
    filtered = []
    
    for image in image_data:
        
        filt_image = filt(image)
        filtered.append(filt_image)
        
    return filtered


for filt in filter_inst:
    
    filtered_images_train.append(use_filters(train_raw_x, filt))
    filtered_images_test.append(use_filters(test_raw_x, filt))
    
        


In [4]:
np.shape(filtered_images_train[0])

(28709, 48, 48)

In [5]:

train_x = []
test_x = []

train_x.append(deepcopy(train_raw_x))
test_x.append(deepcopy(test_raw_x))

for train_array, test_array in zip(filtered_images_train, filtered_images_test):
    
    train_x.append(train_array)
    test_x.append(test_array)
    
    
train_y = deepcopy(train_raw_y)
test_y = deepcopy(test_raw_y)


"""train_x = deepcopy(train_raw_x)
test_x = deepcopy(test_raw_x)

for train_array, test_array in zip(filtered_images_train, filtered_images_test):
    
    train_x = np.concatenate([train_x, train_array])
    test_x = np.concatenate([test_x, test_array])
    
    
train_y = deepcopy(train_raw_y)
test_y = deepcopy(test_raw_y)

for i in range(len(filter_inst)):
    train_y = np.concatenate([train_y, train_raw_y])
    test_y = np.concatenate([test_y, test_raw_y])
    
"""
    



'train_x = deepcopy(train_raw_x)\ntest_x = deepcopy(test_raw_x)\n\nfor train_array, test_array in zip(filtered_images_train, filtered_images_test):\n    \n    train_x = np.concatenate([train_x, train_array])\n    test_x = np.concatenate([test_x, test_array])\n    \n    \ntrain_y = deepcopy(train_raw_y)\ntest_y = deepcopy(test_raw_y)\n\nfor i in range(len(filter_inst)):\n    train_y = np.concatenate([train_y, train_raw_y])\n    test_y = np.concatenate([test_y, test_raw_y])\n    \n'

In [6]:
# pooling


def harf_mean_pooling(image):
    
    means = []
    
    for hori in range(len(image)//2):
        for ver in range(len(image[0])//2):
        
            mean = np.mean(image[hori:hori+len(image)//2][ver:ver+len(image)//2])

            means.append(mean)
        
    return means
        
    




In [7]:

train_x_use = []
test_x_use = []


# normal
for train in train_x:
    tmp_train = []

    for image in train:
        
        tmp_train.append(open_image(image))
    train_x_use.append(tmp_train)
    
    
for test in test_x:
    tmp_test = []
    for image in test:
        tmp_test.append(open_image(image))
        
    test_x_use.append(tmp_test)


# mean pooling
"""for train in train_x:
    
    train_x_use.append(harf_mean_pooling(train))
    
    
for test in test_x:
    test_x_use.append(harf_mean_pooling(test))
"""



'for train in train_x:\n    \n    train_x_use.append(harf_mean_pooling(train))\n    \n    \nfor test in test_x:\n    test_x_use.append(harf_mean_pooling(test))\n'

In [8]:
np.shape(train_x_use)

(5, 28709, 2304)

In [9]:
# def model


model_sobel = lgb.LGBMClassifier(verbose=1)
model_scharr = lgb.LGBMClassifier(verbose=1)
model_prewitt = lgb.LGBMClassifier(verbose=1)
model_roberts = lgb.LGBMClassifier(verbose=1)
model_raw = lgb.LGBMClassifier(verbose=1)

models = [model_sobel, model_scharr, model_prewitt, model_roberts, model_raw]


In [10]:
# fit model

for model, train_data in zip(models, train_x_use):
    
    model.fit(train_data, train_y)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.359647 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 552538
[LightGBM] [Info] Number of data points in the train set: 28709, number of used features: 2304
[LightGBM] [Info] Start training from score -1.972167
[LightGBM] [Info] Start training from score -4.187324
[LightGBM] [Info] Start training from score -1.946956
[LightGBM] [Info] Start training from score -1.381048
[LightGBM] [Info] Start training from score -1.754797
[LightGBM] [Info] Start training from score -1.782364
[LightGBM] [Info] Start training from score -2.203164
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.309517 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 587520
[LightGBM] [Info] Number of data points in the train set: 28709, number of used features: 2304
[LightGBM] [Info] Start traini

In [48]:
# predict
preds = []

for model, test_data in zip(models, test_x_use):
    pred = model.predict(test_data)
    preds.append(pred)

In [49]:
preds.append(test_y)

In [50]:
preds

df_columns = filter_name + ["raw", "ans"]


pred_df = pd.DataFrame(np.array(preds).T, columns=df_columns)


In [51]:
pred_df.loc[:, filter_name+["raw"]]

,sobel,scharr,prewitt,roberts,raw
0,5,0,3,5,3
1,0,0,0,0,3
2,0,0,3,3,0
3,2,5,5,5,0
4,5,3,5,5,2
...,...,...,...,...,...
7173,6,6,6,6,6
7174,6,4,4,4,4
7175,6,6,6,6,6
7176,6,6,6,6,6


In [35]:
mode_df.mode(axis=1)

,0,1,2,3,4
0,0.0,5.0,NaN,NaN,NaN
1,0.0,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN
3,5.0,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...
7173,6.0,NaN,NaN,NaN,NaN
7174,4.0,NaN,NaN,NaN,NaN
7175,6.0,NaN,NaN,NaN,NaN
7176,6.0,NaN,NaN,NaN,NaN


In [52]:

mode_df = pred_df.loc[:, filter_name+["ans"]].mode(axis=1)




In [53]:
mode_df

,0,1,2,3,4
0,0.0,5.0,NaN,NaN,NaN
1,0.0,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN
3,5.0,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...
7173,6.0,NaN,NaN,NaN,NaN
7174,4.0,NaN,NaN,NaN,NaN
7175,6.0,NaN,NaN,NaN,NaN
7176,6.0,NaN,NaN,NaN,NaN


In [54]:
mode_df.columns = filter_name+["raw"]

In [55]:
mode_df

,sobel,scharr,prewitt,roberts,raw
0,0.0,5.0,NaN,NaN,NaN
1,0.0,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN
3,5.0,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...
7173,6.0,NaN,NaN,NaN,NaN
7174,4.0,NaN,NaN,NaN,NaN
7175,6.0,NaN,NaN,NaN,NaN
7176,6.0,NaN,NaN,NaN,NaN


In [18]:
count = 0


for i in range(len(pred_df["ans"])):
    
    if np.any(mode_df.loc[i, :].values == pred_df["ans"][i]):
        count += 1

In [33]:
file = ["sobel","scharr","prewitt","roberts","raw"]
cnt = 0
for i in range(len(pred_df["ans"])):
    for j in file:
        
        

'\nfor i in range(len(pred_df["ans"])):\n    for j in file:\n'

In [64]:
if mode_df['sobel'][0] == "0.0":
    print(1)

In [85]:
mode_x =mode_df.mode(axis=1).count(axis=1)

In [93]:
mode_df.mode(axis=1)
cnt =0
for i in range(len(mode_x)):
    if mode_x[i] ==1:
        if mode_df['sobel'][i] == pred_df["ans"][i]:
            cnt +=1
    

In [84]:
a[4]

1

In [34]:
for i in file:
    print(i)

sobel
scharr
prewitt
roberts
raw


In [95]:
count / len(pred_df)

0.5923655614377263